# Notebook 5: Inferencing with Deployed Model

### In this notebook, we deploy the model previously logged to MLflow and make some predictions to demonstrate that it works. We also transform some of the data in the test set to practice model monitoring in Databricks.

In [0]:
import mlflow
import mlflow.sklearn
import pandas as pd

# ----------------------------
# 1. Load registered model
# ----------------------------
model_name = "rf_ames_model"
model_version = 1 

# Construct the MLflow model URI
model_uri = f"models:/{model_name}/{model_version}"

# Load the model
loaded_model = mlflow.sklearn.load_model(model_uri)
print("Model loaded for inference!")



Model loaded for inference!


In [0]:
# ----------------------------
# 2. Load test data for evaluation
# ----------------------------

# Paths to the saved test set
X_test_path = "/Workspace/Users/iakidwell@uchicago.edu/X_test.csv"
y_test_path = "/Workspace/Users/iakidwell@uchicago.edu/y_test.csv"

# Load test features and target
X_test = pd.read_csv(X_test_path)
y_test = pd.read_csv(y_test_path).squeeze()  # squeeze to convert to Series

# Quick check
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
print(X_test.head())
print(y_test.head())


X_test shape: (586, 53), y_test shape: (586,)
   MS SubClass  Lot Frontage  Lot Area  ...  Mo Sold  Yr Sold  SalePrice
0           70          68.0      5100  ...        6     2008     161000
1          160          21.0      1890  ...        7     2006     116000
2           60          62.0      7162  ...        5     2006     196500
3           20          60.0      8070  ...        8     2007     123600
4           30          50.0      7000  ...        7     2008     126000

[5 rows x 53 columns]
0    11.989166
1    11.661354
2    12.188423
3    11.724814
4    11.744045
Name: SalePrice_log, dtype: float64


In [0]:
import numpy as np
from sklearn.metrics import mean_squared_error

# ----------------------------
# 3. Make predictions on test set
# ----------------------------
preds_log = loaded_model.predict(X_test)

# Invert the log transform to get SalePrice
preds = np.expm1(preds_log)  # converts log(SalePrice+1) back to SalePrice
y_test_actual = np.expm1(y_test)  # convert actual log values back

# Calculate RMSE manually
rmse = np.sqrt(mean_squared_error(y_test_actual, preds))
print(f"Test RMSE on deployed model: {rmse:.2f}")

# Display the first 5 predictions
for i, price in enumerate(preds[:5], 1):
    print(f"Example {i} predicted SalePrice: ${price:.2f}")


Test RMSE on deployed model: 1877.22
Example 1 predicted SalePrice: $161144.92
Example 2 predicted SalePrice: $115959.14
Example 3 predicted SalePrice: $196642.56
Example 4 predicted SalePrice: $123971.65
Example 5 predicted SalePrice: $125313.19


In [0]:
import mlflow
import pandas as pd

# Absolute path for your experiment
experiment_path = "/Users/iakidwell@uchicago.edu/ames_housing_test_eval"

# Create experiment if it doesn't exist
experiment = mlflow.get_experiment_by_name(experiment_path)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_path)
else:
    experiment_id = experiment.experiment_id

# Start a run in this experiment
with mlflow.start_run(run_name="deployed_model_test_eval", experiment_id=experiment_id) as run:
    # Log metric
    mlflow.log_metric("test_rmse", rmse)

    # Save predictions and log as artifact
    eval_df = pd.DataFrame({
        "y_true": y_test_actual,
        "y_pred": preds
    })
    eval_csv = "test_predictions.csv"
    eval_df.to_csv(eval_csv, index=False)
    mlflow.log_artifact(eval_csv)

    print(f"Logged test evaluation to MLflow run {run.info.run_id}")


Logged test evaluation to MLflow run 0136aeb20deb4287b5ce0c72f6be7fc4


# Feature Changes

## This is done to test out the modeling monitoring dashboard in Databricks


In [0]:
# List all features in X_test
print("Features in the dataset:")
for i, col in enumerate(X_test.columns, 1):
    print(f"{i}. {col}")


Features in the dataset:
1. MS SubClass
2. Lot Frontage
3. Lot Area
4. Overall Qual
5. Overall Cond
6. Year Built
7. Year Remod/Add
8. Mas Vnr Area
9. Exter Qual
10. Exter Cond
11. Bsmt Qual
12. Bsmt Cond
13. Bsmt Exposure
14. BsmtFin Type 1
15. BsmtFin SF 1
16. BsmtFin Type 2
17. BsmtFin SF 2
18. Bsmt Unf SF
19. Total Bsmt SF
20. Heating QC
21. 1st Flr SF
22. 2nd Flr SF
23. Low Qual Fin SF
24. Gr Liv Area
25. Bsmt Full Bath
26. Bsmt Half Bath
27. Full Bath
28. Half Bath
29. Bedroom AbvGr
30. Kitchen AbvGr
31. Kitchen Qual
32. TotRms AbvGrd
33. Functional
34. Fireplaces
35. Fireplace Qu
36. Garage Yr Blt
37. Garage Finish
38. Garage Cars
39. Garage Area
40. Garage Qual
41. Garage Cond
42. Paved Drive
43. Wood Deck SF
44. Open Porch SF
45. Enclosed Porch
46. 3Ssn Porch
47. Screen Porch
48. Pool Area
49. Pool QC
50. Misc Val
51. Mo Sold
52. Yr Sold
53. SalePrice


In [0]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import mlflow

# Make a copy of X_test so we don't overwrite original test data
X_test_mutated = X_test.copy()

# Specify the features to modify
feature_1 = "Lot Area"
feature_2 = "Fireplaces"

# Mutate the features: random perturbation
np.random.seed(42)  # for reproducibility
X_test_mutated[feature_1] = X_test_mutated[feature_1] + np.random.normal(0, 1, size=X_test_mutated.shape[0])
X_test_mutated[feature_2] = X_test_mutated[feature_2] + np.random.normal(0, 1, size=X_test_mutated.shape[0])

# Predict with the deployed model
mutated_preds_log = loaded_model.predict(X_test_mutated)
mutated_preds = np.expm1(mutated_preds_log)  # invert log-transform

# Evaluate RMSE on mutated data (manual square root)
y_test_actual = np.expm1(y_test)
mutated_rmse = mean_squared_error(y_test_actual, mutated_preds) ** 0.5
print(f"Test RMSE on mutated test data: {mutated_rmse:.2f}")

# ----------------------------
# Log evaluation with MLflow
# ----------------------------
experiment_path = "/Users/iakidwell@uchicago.edu/ames_housing_test_eval"  # update with your workspace path
mlflow.set_experiment(experiment_path)

with mlflow.start_run(run_name="mutated_test_eval") as run:
    mlflow.log_metric("mutated_test_rmse", mutated_rmse)
    print(f"Logged RMSE to MLflow run {run.info.run_id}")


Test RMSE on mutated test data: 1877.41
Logged RMSE to MLflow run 38834ece25b94b8f943264e34a2c489a
